Nothing works musk depricated the api, RIP Twitter

In [15]:
import requests
from bs4 import BeautifulSoup
import csv
import time

def fetch_tweets(username, num_tweets=200):
    tweets_data = []
    url = f'https://twitter.com/{username}'
    headers = {'User-Agent': 'Mozilla/5.0'}

    while len(tweets_data) < num_tweets:
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')
        tweets = soup.find_all('div', {'data-testid': 'tweet'})

        if not tweets:
            break

        for tweet in tweets:
            if len(tweets_data) >= num_tweets:
                break

            try:
                text = tweet.find('div', {'lang': True}).get_text()
                tweet_id = tweet['data-tweet-id']
                created_at = tweet.find('time')['datetime']
                tweet_data = {
                    'tweet_id': tweet_id,
                    'created_at': created_at,
                    'full_text': text,
                    'url': f'https://twitter.com/{username}/status/{tweet_id}'
                }
                tweets_data.append(tweet_data)
            except:
                continue

        last_tweet = tweets[-1]
        last_tweet_id = last_tweet['data-tweet-id']
        url = f'https://twitter.com/{username}/max_id/{last_tweet_id}'

    return tweets_data

username = 'fampayindia'
tweets_data = fetch_tweets(username)

with open('fampay_twitter_data.csv', 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['tweet_id', 'created_at', 'full_text', 'url']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for tweet_data in tweets_data:
        writer.writerow(tweet_data)

print("FamPay Twitter data has been saved to fampay_twitter_data.csv.")
